In [1]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import pickle
import re
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('all')

In [2]:
# Utility functions
def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

def flatten(t):
    return [item for sublist in t for item in sublist]

def lower(pharse: str) -> str:
    return pharse.lower()
    
reports = get_all_filenames()
reports = [r for r in reports if r.endswith('.txt')]
reports = [r for r in reports if r.startswith('Darlloz')]
# reports.remove('Dofloo-360InternetSecurityCenter.txt')
print(reports)

['Darlloz-Broadcom.txt', 'Darlloz-Giac.txt', 'Darlloz-Icc17.txt', 'Darlloz-ResearchCheckPoint.txt', 'Darlloz-Trendmicro.txt', 'Darlloz-Trendmicro2.txt', 'Darlloz-cisco.txt']


將所有 report 斷句，另存csv檔  
- 有人為斷句的文章: `Dofloo-Trendmicro.txt`

In [3]:
Path("./sentence csvs").mkdir(parents=True, exist_ok=True)

for rname in reports:
    with open(rname, 'r', encoding="utf-8") as f:
        raw_text = f.read()
    # raw_text = lower(raw_text)
    text = re.sub(r"\"", '', raw_text)
    text_sentences = sent_tokenize(text, language='english')

    must_split_ids = [] # ~~ 符號表示人為強制斷句，若文本中有 ~~ 則表示nltk斷得不夠徹底需再進行強制斷句
    if text.find('~~') != -1:
        tmp = text_sentences
        text_sentences = []
        for i, sent in enumerate(tmp): 
            try:
                text_sentences.extend(sent.split('~~'))
            except:
                text_sentences.append(sent)
        # print(text_sentences)
    
    [sent.strip() for sent in text_sentences] # build up csv file
    lst = []
    for i, sent in enumerate(text_sentences):
        lst.append([f'Sentence {i+1}', sent])
    rname_wo_txt = rname.split('.')[0] # 去掉 .txt
    df = pd.DataFrame(lst, columns=['Number','Content'])
    df.to_csv(f'./sentence csvs/{rname_wo_txt}.csv', index=False)

測試將一篇 report  斷句

In [7]:
with open(reports[2], 'r', encoding="utf-8") as f:
    raw_text = f.read()
# raw_text = '''"When you start coding, it makes you feel smart in itself, like you're in the Matrix [film]," says Janine Luk, a 26 year-old software engineer who works in London. Born in Hong Kong, she started her career in yacht marketing in the south of France but found is's "a bit repetitive and superficial". So, she started teaching herself to code after work, followed by a 15-week coding boot camp.'''

# removing double quotes
text = re.sub(r"\"", '', raw_text)

# breaking text into sentences
text_sentences = sent_tokenize(text, language='english')

# printing out sentences
for i, sent in enumerate(text_sentences):
    print("Sentence {}: {}".format(i + 1, sent), end = "\n\n")

Sentence 1: AESDDoS bot exploits CVE-2019-3396 flaw to hit Atlassian Confluence Server
April 28, 2019  By Pierluigi Paganini

A new variant of the AESDDoS bot is exploiting a recent vulnerability in the Atlassian collaborative software Confluence.

Sentence 2: Security experts at Trend Micro have spotted a new variant of AESDDoS botnet that is exploiting a recently discovered vulnerability in the Atlassian collaborative software Confluence.

Sentence 3: The flaw exploited in the attacks, tracked as CVE-2019-3396, is a server-side template injection vulnerability that resides in the Widget Connector macro in Confluence Server.

Sentence 4: Threat actors leverage the vulnerability to install denial of service (DDoS) malware and crypto-currency miners, and to remotely execute code.

Sentence 5: “In our analysis, we saw that an attacker was able to exploit CVE-2019-3396 to infect machines with the AESDDoS botnet malware.” reads the analysis published by Trend Micro.

Sentence 6: “A shell c